# **Installing pinecone**

---



In [13]:
!pip install --upgrade pinecone-client

# **Installing pre-trained Transformers for better and easy implementation**

---



In [14]:
!pip install sentence-transformers

# **Importing necessary libraries**

---


In [15]:
from sentence_transformers import SentenceTransformer
import numpy as np
from pinecone import Pinecone, ServerlessSpec

# **Defining Pinecone**

---



In [16]:
pc = Pinecone(api_key="1fa98373-be29-457c-8dbf-777414f715d1")

index_name = "my-index"
if index_name not in pc.list_indexes().names():
  # try:
    pc.create_index(
        name=index_name,
        dimension=384,
        metric="euclidean",
        spec=ServerlessSpec(cloud="aws", region="us-east-1")
    )
  # except TypeError:
  #    pass  # Ignore the TypeError for now

index = pc.Index(index_name)

# **Importing Document**

---
Not using file handling to import document because it is colab environment


In [17]:
documents=['''
Introduction
Artificial Intelligence (AI) refers to the simulation of human intelligence in machines that are programmed to think like humans and mimic their actions. The term may also be applied to any machine that exhibits traits associated with a human mind such as learning and problem-solving.
''',
'''Types of AI

Narrow AI (Weak AI)
Designed and trained for a particular task. Examples include voice assistants like Siri and Alexa, recommendation systems on streaming services, and self-driving cars.

General AI (Strong AI)
Possesses generalized human cognitive abilities. When presented with an unfamiliar task, a strong AI system can find a solution without human intervention.

Superintelligent AI
Surpasses human intelligence and capability in all fields, including creativity, general wisdom, and social skills.
''',
'''History of AI

1950s: Alan Turing proposed the "Turing Test" to determine if a machine can exhibit intelligent behavior indistinguishable from a human.
1956: The term "Artificial Intelligence" was coined by John McCarthy during the Dartmouth Conference.
1960s-1970s: Development of early AI programs like ELIZA (a simple chatbot) and the first AI-based chess game.
1980s: Introduction of machine learning, where systems could learn and improve from experience.
1997: IBM's Deep Blue defeated world chess champion Garry Kasparov.
2010s: Advancements in deep learning and neural networks led to significant improvements in AI capabilities.
''',
'''Key Concepts and Technologies in AI

Machine Learning (ML)
A subset of AI that involves the use of algorithms and statistical models to perform specific tasks without explicit instructions, relying on patterns and inference instead.

Deep Learning
A subset of ML that uses neural networks with many layers (hence "deep") to analyze various factors of data.

Neural Networks
Inspired by the human brain, these networks consist of neurons (nodes) and are designed to recognize patterns.

Natural Language Processing (NLP)
Enables machines to understand, interpret, and respond to human language. Examples include chatbots, sentiment analysis, and language translation.

Computer Vision
Allows machines to interpret and make decisions based on visual input. Applications include facial recognition, autonomous driving, and medical image analysis.

Robotics
Involves designing and creating robots that can perform tasks autonomously or semi-autonomously. Examples include industrial robots, drones, and robotic vacuum cleaners.

Reinforcement Learning
A type of ML where an agent learns to make decisions by performing actions and receiving rewards or penalties.
''',
'''Applications of AI

Healthcare
AI is used in medical imaging, drug discovery, personalized medicine, and virtual health assistants.

Finance
AI algorithms are employed in fraud detection, algorithmic trading, personalized banking, and risk management.

Transportation
Autonomous vehicles, traffic management systems, and logistics optimization.

Retail
Recommendation systems, customer service chatbots, inventory management, and personalized marketing.

Entertainment
Content recommendation (Netflix, Spotify), game development, and virtual influencers.

Manufacturing
Predictive maintenance, quality control, and robotics automation.

Education
Adaptive learning platforms, automated grading, and AI tutors.

Agriculture
Precision farming, crop monitoring, and automated harvesting.
''',
'''
Challenges in AI

Ethical Considerations
Bias and fairness, privacy concerns, and the potential for misuse.

Job Displacement
Automation of tasks leading to potential unemployment.

Security
AI systems can be vulnerable to hacking and other security threats.

Transparency and Accountability
Understanding and explaining how AI models make decisions.

Regulation
Establishing guidelines and laws to ensure the safe and ethical use of AI.
''',
'''
Future of AI

Advancements in AI Research
Continued progress in areas like general AI, human-AI collaboration, and AI safety.

AI in Daily Life
Increased integration of AI in everyday devices and services.

AI and Society
Ongoing discussions about the societal impact of AI, including its role in decision-making and governance.
''',
'''
Conclusion
AI is transforming various sectors and reshaping the world as we know it. While it offers tremendous benefits, it also presents significant challenges that must be addressed to ensure its responsible and ethical development. Understanding AI's capabilities, applications, and potential impacts is crucial for navigating its future.''']

# **Defining SentenceTransformer Model**

---



In [18]:
Model=SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

# **Embedding of Document**

---



In [19]:
embeddings=Model.encode(documents)

In [20]:
for doc_id, embedding in enumerate(embeddings):
    index.upsert([(str(doc_id), embedding)])

# **Embedding Query string**

---



In [21]:
query='what is AI' # input('enter you Query here: ')
query_embedding=Model.encode([query])[0]

query_embedding = query_embedding.tolist()    # Converting numpy array to List

results=index.query(vector=query_embedding, top_k=5)         # Retrieving top 5 most similar documents

retrieved_documents=[documents[int(res['id'])] for res in results['matches']]

In [22]:
retrieved_documents

['\nIntroduction\nArtificial Intelligence (AI) refers to the simulation of human intelligence in machines that are programmed to think like humans and mimic their actions. The term may also be applied to any machine that exhibits traits associated with a human mind such as learning and problem-solving.\n',
 'Applications of AI\n\nHealthcare\nAI is used in medical imaging, drug discovery, personalized medicine, and virtual health assistants.\n\nFinance\nAI algorithms are employed in fraud detection, algorithmic trading, personalized banking, and risk management.\n\nTransportation\nAutonomous vehicles, traffic management systems, and logistics optimization.\n\nRetail\nRecommendation systems, customer service chatbots, inventory management, and personalized marketing.\n\nEntertainment\nContent recommendation (Netflix, Spotify), game development, and virtual influencers.\n\nManufacturing\nPredictive maintenance, quality control, and robotics automation.\n\nEducation\nAdaptive learning plat

# **Installing (GPT2LMHeadModel & GPT2Tokenizer) for pretrained models**

---



In [23]:
!pip install transformers
from transformers import GPT2LMHeadModel, GPT2Tokenizer

# **Generating texts from GPT model**

---

Ensuring to split Unecessary informations. like Retrived document data's and else.

In [24]:
model_name='gpt2'
tokenizer=GPT2Tokenizer.from_pretrained(model_name)
model=GPT2LMHeadModel.from_pretrained(model_name)

context=" ".join(retrieved_documents)
prompt=f"Based on the following information: {context}\n\n split. Question: {query}\nAnswer (in a concise and clear manner):"

inputs=tokenizer.encode(prompt,return_tensors='pt',max_length=512, truncation=True)
outputs=model.generate(
    inputs,max_length=512,num_return_sequences=1,no_repeat_ngram_size=2,do_sample=True,top_p=0.95,top_k=50
)

response=tokenizer.decode(outputs[0],skip_special_tokens=True)

if "split." in response:
    answer=response.split("split.")[1].strip()
else:
    answer=response.strip()

# Post-process to remove any extraneous text or sentences
# answer=answer.split('.')[0] + '.'
print(answer)


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Question: what is AI
Answer (in a concise and clear manner): AI is an artificial intelligence that advances technology to recognize human and other forms of behavior, such that human consciousness evolves to become a superior
